ENVIRONMENT SETTING

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
import pandas as pd

In [3]:
import time
import urllib
from tqdm import tqdm, trange

In [4]:
tqdm.pandas()

In [5]:
options = webdriver.ChromeOptions()
# options.add_argument('no-sandbox')
# options.add_argument('headless')
# options.add_argument('window-size=1920x1080')
# options.add_argument('disable-gpu')

GET LINK

In [6]:
# NeurIPS 2019는 딱히 데이터가 느낌있게 나와주지 않음.

accepted_links_sets = [
    [                                                                                               ##### ICLR 2023
        'https://openreview.net/group?id=ICLR.cc/2023/Conference#notable-top-5-',                   # Notable 5% (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2023/Conference#notable-top-25-',                  # Notable 25% (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2023/Conference#poster',                           # Paper (Accepted)
    ],
    [                                                                                               ##### ICLR 2022
        'https://openreview.net/group?id=ICLR.cc/2022/Conference#oral-submissions',                 # Oral Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2022/Conference#spotlight-submissions',            # Spotlight Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2022/Conference#poster-submissions',               # Poster Presentations (Accepted)
    ],
    [                                                                                               ##### ICLR 2021
        'https://openreview.net/group?id=ICLR.cc/2021/Conference#oral-presentations',               # Oral Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2021/Conference#spotlight-presentations',          # Spotlight Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2021/Conference#poster-presentations',             # Poster Presentations (Accepted)
    ],
    [                                                                                               ##### ICLR 2020
        'https://openreview.net/group?id=ICLR.cc/2020/Conference#poster-presentations',             # Poster Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2020/Conference#spotlight-presentations',          # Spotlight Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2020/Conference#oral-presentations',               # Oral Presentations (Accepted)
    ],
    [                                                                                               ##### ICLR 2019
        'https://openreview.net/group?id=ICLR.cc/2019/Conference#poster-presentations',             # Poster Presentations (Accepted)
        'https://openreview.net/group?id=ICLR.cc/2019/Conference#oral-presentations',               # Oral Presentations (Accepted)
    ]
]

rejected_links = [
    'https://openreview.net/group?id=ICLR.cc/2023/Conference#submitted',                            # ICLR 2023
    'https://openreview.net/group?id=ICLR.cc/2022/Conference#submitted-submissions',                # ICLR 2022
    'https://openreview.net/group?id=ICLR.cc/2021/Conference#withdrawn-rejected-submissions',       # ICLR 2021
    'https://openreview.net/group?id=ICLR.cc/2020/Conference#withdrawn-rejected-submissions',       # ICLR 2020
    'https://openreview.net/group?id=ICLR.cc/2019/Conference#withdrawn-rejected-submissions',       # ICLR 2019
]

conference_names = [
    'ICLR_2023',
    'ICLR_2022',
    'ICLR_2021',
    'ICLR_2020',
    'ICLR_2019',
    # 'NeurIPS_2022',
    # 'NeurIPS_2021',
]

In [7]:
def collect_links(link):
    df = pd.DataFrame(columns = ['title', 'link'])
    
    driver = webdriver.Chrome(service = Service(executable_path = './driver/chromedriver'), options = options)
    time.sleep(5)
    driver.get(link)
    while len(driver.find_elements(By.CSS_SELECTOR, 'li.note h4 a')) == 0:
        time.sleep(1)

    try:
        num_pages = int([button for button in driver.find_elements(By.CSS_SELECTOR, 'nav > ul.pagination > li') if button.text == '»'][0].get_attribute('data-page-number'))
    except:
        num_pages = 1

    for page in trange(num_pages):
        # Get New Paper Lists
        new_papers = pd.DataFrame(data = [{'title': x.text, 'link': x.get_attribute('href')} for x in driver.find_elements(By.CSS_SELECTOR, 'li.note h4 a') if x.text != ''])
        df = pd.concat([df, new_papers]).reset_index(drop = True)

        # Click Next Button
        if page < num_pages - 1:
            [button for button in driver.find_elements(By.CSS_SELECTOR, 'nav > ul.pagination > li > a') if button.text == '›'][0].click()

        # Sleep
        time.sleep(10)

    driver.close()
    return df

In [8]:
for name, accept_links, reject_link in zip(conference_names, accepted_links_sets, rejected_links):
    print(f'Crawling from {name}')
    print('-----------------------')
    
    accept_df = pd.DataFrame(columns = ['title', 'link'])

    for accept_link in accept_links:
        print(f'- Accept Link: {accept_link}')
        accept_df = pd.concat([accept_df, collect_links(accept_link)], axis = 0)
    
    print(f'- Reject Link: {reject_link}')
    reject_df = collect_links(reject_link)

    accept_df = accept_df.reset_index(drop = True)
    reject_df = reject_df.reset_index(drop = True)

    accept_df.to_csv(f'./data/conferences/{name}-accept.csv')
    reject_df.to_csv(f'./data/conferences/{name}-reject.csv')

    print()

Crawling from ICLR_2023
-----------------------
- Accept Link: https://openreview.net/group?id=ICLR.cc/2023/Conference#notable-top-5-


100%|██████████| 4/4 [00:46<00:00, 11.59s/it]


- Accept Link: https://openreview.net/group?id=ICLR.cc/2023/Conference#notable-top-25-


100%|██████████| 12/12 [02:19<00:00, 11.63s/it]


- Accept Link: https://openreview.net/group?id=ICLR.cc/2023/Conference#poster


...

100%|██████████| 1/1 [00:19<00:00, 19.32s/it]


- Accept Link: https://openreview.net/group?id=ICLR.cc/2019/Conference#oral-presentations


100%|██████████| 1/1 [00:10<00:00, 10.51s/it]


- Reject Link: https://openreview.net/group?id=ICLR.cc/2019/Conference#withdrawn-rejected-submissions


100%|██████████| 1/1 [00:31<00:00, 31.01s/it]